In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Bahia - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_BA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego,Bahia - Consumo de Cimento (t)
0,2003-1,0.299858,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.669899,39.798880,1.317344e+08,8.384593e+06,8.566149,1.216359e+08,8.348779,151.297
1,2003-2,0.301903,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.670210,39.480034,1.318964e+08,8.391946e+06,8.569210,1.216914e+08,8.342979,138.707
2,2003-3,0.303709,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.670521,39.400256,1.320584e+08,8.399299e+06,8.572270,1.217469e+08,8.337179,135.009
3,2003-4,0.305311,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.670831,39.417185,1.322204e+08,8.406652e+06,8.575331,1.218023e+08,8.331379,126.554
4,2003-5,0.306860,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.671142,39.479943,1.323824e+08,8.414005e+06,8.578392,1.218578e+08,8.325579,137.331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,0.597113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.069163,NaN,NaN,NaN,NaN,NaN,366.305
236,2022-9,0.596178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.752943,NaN,NaN,NaN,NaN,NaN,346.042
237,2022-10,0.594662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.537361,NaN,NaN,NaN,NaN,NaN,347.901
238,2022-11,0.592436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.971241,NaN,NaN,NaN,NaN,NaN,310.845


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
0,-1.440329,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.143224,-1.723080,-1.703176,-0.816345,-2.235724,-2.158201,-0.884052
1,-1.404914,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.108465,-1.735985,-1.684201,-0.772933,-2.194204,-2.117854,-0.885724
2,-1.373637,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.073707,-1.739214,-1.665225,-0.729522,-2.152684,-2.077508,-0.887395
3,-1.345888,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.038948,-1.738529,-1.646249,-0.686111,-2.111164,-2.037161,-0.889067
4,-1.319065,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.004190,-1.735989,-1.627274,-0.642700,-2.069645,-1.996814,-0.890739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.796141,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.261516,0.511535,1.096947,-1.668064,0.877256,0.724962,1.200853
188,1.811284,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.253736,0.513264,1.085441,-1.663847,0.851021,0.704849,1.200487
189,1.833479,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.245956,0.526782,1.073935,-1.659631,0.824786,0.684737,1.200122
190,1.853109,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.238176,0.525379,1.062429,-1.655414,0.798552,0.664624,1.199756


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      129.857
1      126.813
2      152.225
3      136.288
4      148.117
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Bahia - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
0,-1.440329,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.143224,-1.723080,-1.703176,-0.816345,-2.235724,-2.158201,-0.884052
1,-1.404914,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.108465,-1.735985,-1.684201,-0.772933,-2.194204,-2.117854,-0.885724
2,-1.373637,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.073707,-1.739214,-1.665225,-0.729522,-2.152684,-2.077508,-0.887395
3,-1.345888,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.038948,-1.738529,-1.646249,-0.686111,-2.111164,-2.037161,-0.889067
4,-1.319065,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.004190,-1.735989,-1.627274,-0.642700,-2.069645,-1.996814,-0.890739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.097864,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.430739,0.617097,1.205171,-1.332406,1.004714,1.051323,1.273469
158,1.120412,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.428086,0.590103,1.206522,-1.357999,1.007770,1.045502,1.268173
159,1.139591,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.425433,0.561454,1.207873,-1.383592,1.010826,1.039682,1.262877
160,1.156903,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.422780,0.523492,1.209225,-1.409185,1.013881,1.033862,1.257581


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      129.857
1      126.813
2      152.225
3      136.288
4      148.117
        ...   
157    213.266
158    285.938
159    219.576
160    267.203
161    240.714
Name: Bahia - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
126,0.558660,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.461215,1.205505,0.833268,0.194482,0.840990,1.014636,0.957509
127,0.586295,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.471044,1.203037,0.852943,0.147537,0.856945,1.027926,0.975019
128,0.614441,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.480874,1.178322,0.872618,0.100591,0.872900,1.041216,0.992528
129,0.639726,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.490704,1.155459,0.892293,0.053646,0.888855,1.054506,1.010038
130,0.666428,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.500533,1.124580,0.911968,0.006700,0.904809,1.067796,1.027547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.796141,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.261516,0.511535,1.096947,-1.668064,0.877256,0.724962,1.200853
188,1.811284,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.253736,0.513264,1.085441,-1.663847,0.851021,0.704849,1.200487
189,1.833479,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.245956,0.526782,1.073935,-1.659631,0.824786,0.684737,1.200122
190,1.853109,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.238176,0.525379,1.062429,-1.655414,0.798552,0.664624,1.199756


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1948399246, 95928480, 3995884774, 2067311376, 299584988, 4102392426, 455557272, 4172311529, 490057157, 3392208108, 99816580, 2034207888, 812802147, 3933679052, 2369192805, 698079608, 2081959370, 3460374243, 2970741742, 3496608837, 97133546, 1437914059, 2696415176, 1080601799, 3411830636, 3126418624, 216580950, 2229567040, 454472537, 887500047, 1573472394, 569524816, 3271148105, 1341169542, 2608077165, 2010971921, 823893208, 1180011543, 154123279, 3244036765, 2212411522, 2991022445, 4041869319, 3442641308, 2730717607, 4237241218, 3658474634, 3464753917, 1065069430, 1800465589]


Step: 0 ___________________________________________
val_loss: 557.7199096679688
winner_seed: 1948399246


Step: 1 ___________________________________________
val_loss: 551.3668823242188
winner_seed: 95928480


Step: 2 ___________________________________________
val_loss: 432.91693115234375
winner_seed: 3995884774


Step: 3 ___________________________________________
val_loss: 578.5377197265625


Step: 4 _______

In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 201ms/step - loss: 67399.6172 - val_loss: 12884.6465
Epoch 2/10000
4/4 [==============================] - 0s 10ms/step - loss: 5291.6450 - val_loss: 1858.8279
Epoch 3/10000
4/4 [==============================] - 0s 9ms/step - loss: 1453.5789 - val_loss: 1235.6108
Epoch 4/10000
4/4 [==============================] - 0s 9ms/step - loss: 1308.7985 - val_loss: 5434.4609
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 1811.9977 - val_loss: 1243.5170
Epoch 6/10000
4/4 [==============================] - 0s 9ms/step - loss: 877.3895 - val_loss: 1730.1909
Epoch 7/10000
4/4 [==============================] - 0s 9ms/step - loss: 842.6059 - val_loss: 882.0933
Epoch 8/10000
4/4 [==============================] - 0s 9ms/step - loss: 638.4999 - val_loss: 746.0182
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 764.9133 - val_loss: 699.4972
Epoch 10/10000
4/4 [==============================] - 0s 

4/4 [==============================] - 0s 9ms/step - loss: 327.2115 - val_loss: 778.4673
Epoch 80/10000
4/4 [==============================] - 0s 9ms/step - loss: 531.6571 - val_loss: 686.9511
Epoch 81/10000
4/4 [==============================] - 0s 9ms/step - loss: 409.7985 - val_loss: 719.6722
Epoch 82/10000
4/4 [==============================] - 0s 9ms/step - loss: 329.8363 - val_loss: 633.1568
Epoch 83/10000
4/4 [==============================] - 0s 9ms/step - loss: 317.2296 - val_loss: 725.4407
Epoch 84/10000
4/4 [==============================] - 0s 9ms/step - loss: 320.3624 - val_loss: 662.4492
Epoch 85/10000
4/4 [==============================] - 0s 9ms/step - loss: 256.0253 - val_loss: 711.7719
Epoch 86/10000
4/4 [==============================] - 0s 9ms/step - loss: 268.1018 - val_loss: 701.9041
Epoch 87/10000
4/4 [==============================] - 0s 9ms/step - loss: 306.9902 - val_loss: 654.9179
Epoch 88/10000
4/4 [==============================] - 0s 9ms/step - loss: 318.4

4/4 [==============================] - 0s 9ms/step - loss: 232.0002 - val_loss: 728.4541
Epoch 158/10000
4/4 [==============================] - 0s 9ms/step - loss: 220.1079 - val_loss: 665.2841
Epoch 159/10000
4/4 [==============================] - 0s 9ms/step - loss: 200.2802 - val_loss: 638.5925
Epoch 160/10000
4/4 [==============================] - 0s 9ms/step - loss: 222.0802 - val_loss: 644.6259
Epoch 161/10000
4/4 [==============================] - 0s 9ms/step - loss: 212.3907 - val_loss: 638.0886
Epoch 162/10000
4/4 [==============================] - 0s 9ms/step - loss: 209.3170 - val_loss: 677.2300
Epoch 163/10000
4/4 [==============================] - 0s 9ms/step - loss: 236.2408 - val_loss: 820.5177
Epoch 164/10000
4/4 [==============================] - 0s 9ms/step - loss: 281.0984 - val_loss: 730.3291
Epoch 165/10000
4/4 [==============================] - 0s 9ms/step - loss: 217.2528 - val_loss: 696.9658
Epoch 166/10000
4/4 [==============================] - 0s 9ms/step - lo

4/4 [==============================] - 0s 9ms/step - loss: 260.0697 - val_loss: 561.5042
Epoch 236/10000
4/4 [==============================] - 0s 9ms/step - loss: 284.7995 - val_loss: 646.4047
Epoch 237/10000
4/4 [==============================] - 0s 9ms/step - loss: 240.8978 - val_loss: 710.6550
Epoch 238/10000
4/4 [==============================] - 0s 9ms/step - loss: 240.7895 - val_loss: 660.6215
Epoch 239/10000
4/4 [==============================] - 0s 9ms/step - loss: 248.5210 - val_loss: 673.3228
Epoch 240/10000
4/4 [==============================] - 0s 9ms/step - loss: 239.1759 - val_loss: 655.7102
Epoch 241/10000
4/4 [==============================] - 0s 9ms/step - loss: 194.0923 - val_loss: 660.9221
Epoch 242/10000
4/4 [==============================] - 0s 9ms/step - loss: 216.8013 - val_loss: 670.3974
Epoch 243/10000
4/4 [==============================] - 0s 9ms/step - loss: 200.5100 - val_loss: 686.1619
Epoch 244/10000
4/4 [==============================] - 0s 9ms/step - lo

4/4 [==============================] - 0s 9ms/step - loss: 301.1874 - val_loss: 699.1073
Epoch 314/10000
4/4 [==============================] - 0s 9ms/step - loss: 240.5060 - val_loss: 770.5722
Epoch 315/10000
4/4 [==============================] - 0s 9ms/step - loss: 240.1859 - val_loss: 722.9952
Epoch 316/10000
4/4 [==============================] - 0s 9ms/step - loss: 249.8279 - val_loss: 719.7623
Epoch 317/10000
4/4 [==============================] - 0s 9ms/step - loss: 222.8681 - val_loss: 716.5412
Epoch 318/10000
4/4 [==============================] - 0s 9ms/step - loss: 218.4409 - val_loss: 731.2611
Epoch 319/10000
4/4 [==============================] - 0s 9ms/step - loss: 246.4418 - val_loss: 695.0532
Epoch 320/10000
4/4 [==============================] - 0s 9ms/step - loss: 209.4030 - val_loss: 669.5845
Epoch 321/10000
4/4 [==============================] - 0s 9ms/step - loss: 204.9156 - val_loss: 679.6083
Epoch 322/10000
4/4 [==============================] - 0s 9ms/step - lo

...

4/4 [==============================] - 0s 9ms/step - loss: 110.9673 - val_loss: 716.1533
Epoch 1247/10000
4/4 [==============================] - 0s 9ms/step - loss: 129.0612 - val_loss: 615.0714
Epoch 1248/10000
4/4 [==============================] - 0s 9ms/step - loss: 124.3913 - val_loss: 619.8097
Epoch 1249/10000
4/4 [==============================] - 0s 9ms/step - loss: 136.1626 - val_loss: 635.4084
Epoch 1250/10000
4/4 [==============================] - 0s 9ms/step - loss: 135.8496 - val_loss: 627.8771
Epoch 1251/10000
4/4 [==============================] - 0s 9ms/step - loss: 135.5454 - val_loss: 630.3370
Epoch 1252/10000
4/4 [==============================] - 0s 9ms/step - loss: 133.2803 - val_loss: 623.2960
Epoch 1253/10000
4/4 [==============================] - 0s 9ms/step - loss: 137.7161 - val_loss: 662.6499
Epoch 1254/10000
4/4 [==============================] - 0s 9ms/step - loss: 132.8000 - val_loss: 659.2794
Epoch 1255/10000
4/4 [==============================] - 0s 9ms/

4/4 [==============================] - 0s 9ms/step - loss: 115.5996 - val_loss: 780.0608
Epoch 1324/10000
4/4 [==============================] - 0s 9ms/step - loss: 139.1179 - val_loss: 680.9836
Epoch 1325/10000
4/4 [==============================] - 0s 9ms/step - loss: 109.3761 - val_loss: 703.4608
Epoch 1326/10000
4/4 [==============================] - 0s 9ms/step - loss: 117.0517 - val_loss: 701.1971
Epoch 1327/10000
4/4 [==============================] - 0s 9ms/step - loss: 119.7212 - val_loss: 686.9144
Epoch 1328/10000
4/4 [==============================] - 0s 9ms/step - loss: 113.9350 - val_loss: 666.5095
Epoch 1329/10000
4/4 [==============================] - 0s 9ms/step - loss: 129.0348 - val_loss: 717.1093
Epoch 1330/10000
4/4 [==============================] - 0s 9ms/step - loss: 133.3418 - val_loss: 675.6229
Epoch 1331/10000
4/4 [==============================] - 0s 9ms/step - loss: 116.1642 - val_loss: 662.6406
Epoch 1332/10000
4/4 [==============================] - 0s 9ms/

4/4 [==============================] - 0s 9ms/step - loss: 169.4176 - val_loss: 693.3112
Epoch 1401/10000
4/4 [==============================] - 0s 9ms/step - loss: 160.7411 - val_loss: 721.6055
Epoch 1402/10000
4/4 [==============================] - 0s 9ms/step - loss: 151.2311 - val_loss: 697.8661
Epoch 1403/10000
4/4 [==============================] - 0s 9ms/step - loss: 164.5636 - val_loss: 825.4392
Epoch 1404/10000
4/4 [==============================] - 0s 9ms/step - loss: 163.1666 - val_loss: 869.0689
Epoch 1405/10000
4/4 [==============================] - 0s 9ms/step - loss: 147.3962 - val_loss: 771.1536
Epoch 1406/10000
4/4 [==============================] - 0s 9ms/step - loss: 158.8950 - val_loss: 1141.9982
Epoch 1407/10000
4/4 [==============================] - 0s 9ms/step - loss: 280.1379 - val_loss: 650.5029
Epoch 1408/10000
4/4 [==============================] - 0s 9ms/step - loss: 198.0406 - val_loss: 713.0145
Epoch 1409/10000
4/4 [==============================] - 0s 9ms

4/4 [==============================] - 0s 9ms/step - loss: 151.8952 - val_loss: 781.8203
Epoch 1478/10000
4/4 [==============================] - 0s 9ms/step - loss: 163.6688 - val_loss: 705.6828
Epoch 1479/10000
4/4 [==============================] - 0s 9ms/step - loss: 145.5991 - val_loss: 659.5768
Epoch 1480/10000
4/4 [==============================] - 0s 9ms/step - loss: 175.3807 - val_loss: 656.5201
Epoch 1481/10000
4/4 [==============================] - 0s 9ms/step - loss: 153.6196 - val_loss: 643.8386
Epoch 1482/10000
4/4 [==============================] - 0s 9ms/step - loss: 187.3926 - val_loss: 632.2599
Epoch 1483/10000
4/4 [==============================] - 0s 9ms/step - loss: 144.6637 - val_loss: 638.8969
Epoch 1484/10000
4/4 [==============================] - 0s 9ms/step - loss: 218.2017 - val_loss: 686.4843
Epoch 1485/10000
4/4 [==============================] - 0s 9ms/step - loss: 180.2549 - val_loss: 668.8083
Epoch 1486/10000
4/4 [==============================] - 0s 9ms/

4/4 [==============================] - 0s 9ms/step - loss: 142.0686 - val_loss: 801.4581
Epoch 1555/10000
4/4 [==============================] - 0s 9ms/step - loss: 193.0295 - val_loss: 715.9572
Epoch 1556/10000
4/4 [==============================] - 0s 9ms/step - loss: 140.6282 - val_loss: 726.8337
Epoch 1557/10000
4/4 [==============================] - 0s 9ms/step - loss: 141.4574 - val_loss: 716.2199
Epoch 1558/10000
4/4 [==============================] - 0s 9ms/step - loss: 149.8521 - val_loss: 881.7550
Epoch 1559/10000
4/4 [==============================] - 0s 9ms/step - loss: 149.8936 - val_loss: 809.7810
Epoch 1560/10000
4/4 [==============================] - 0s 9ms/step - loss: 157.8652 - val_loss: 728.9886
Epoch 1561/10000
4/4 [==============================] - 0s 9ms/step - loss: 142.2435 - val_loss: 699.7296
Epoch 1562/10000
4/4 [==============================] - 0s 9ms/step - loss: 132.0895 - val_loss: 711.0947
Epoch 1563/10000
4/4 [==============================] - 0s 9ms/

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,252.091721,252.199722,252.164261,252.003159,251.993347,252.012344,242.628708,237.715622,236.897446,235.841522,234.964142,234.648941,233.946091,235.667572,236.074829,236.148788,236.285416,236.322525,236.467026,236.609238,236.52095,236.554306,236.578506,236.592728,236.658127,236.601089,236.601486,236.539062,236.574142,236.506546
Target,250.101,277.528,270.092,278.146,257.458,231.748,268.336,223.453,241.464,238.901,191.989,272.452,261.009,292.688,258.881,276.879,255.774,208.326,291.428,249.43,241.612,252.303,281.912,200.213,270.511,281.466,285.535,328.259,298.078,264.838
Error,1.990723,25.328293,17.92775,26.142838,5.464661,20.264343,25.707291,14.262619,4.566559,3.059479,42.975143,37.803055,27.062912,57.020416,22.806183,40.730209,19.488586,27.996521,54.960983,12.820755,5.091049,15.748688,45.333481,36.37973,33.852859,44.864914,48.933517,91.71994,61.50386,28.331467


In [38]:
display(mae)
display(mape)

30.004627

0.112703994

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined